In [55]:
import pandas as pd

# Beispiel: Lesen der Daten aus einer Datei
# Pfad zur Datei
file_path = r"C:\Users\bernh\Downloads\drive-download-20240908T150007Z-001\WitMotion New Software\WitMotion(V2024.8.12.4)\Record\2024-09-15\09-51-45-747\data_0.txt"
# "C:\Users\bernh\Downloads\drive-download-20240908T150007Z-001\WitMotion New Software\WitMotion(V2024.8.12.4)\Record\2024-09-15\09-51-45-747\data_0.txt"
# "C:\Users\bernh\Downloads\drive-download-20240908T150007Z-001\WitMotion New Software\WitMotion(V2024.8.12.4)\Record\2024-09-15\05-11-18-527\data_0.txt"
# Daten in den DataFrame laden, mit Tabulator als Trennzeichen
df = pd.read_csv(file_path, sep='\t', index_col=False)


# Überprüfen der Spaltennamen nach dem Zurücksetzen des Index
# print("Spaltennamen nach Index-Zurücksetzung:", df.columns)

# Die ersten paar Zeilen nach der Anpassung anzeigen
print(df.head())

#_# Ersetzen von Kommas durch Punkte in spezifischen Spalten
#_df['Chip Time()'] = df['Chip Time()'].str.replace(',', '.').astype(float)
#_# ACCELERATION:
df['Acceleration X(g)'] = df['Acceleration X(g)'].str.replace(',', '.').astype(float)
df['Acceleration Y(g)'] = df['Acceleration Y(g)'].str.replace(',', '.').astype(float)
df['Acceleration Z(g)'] = df['Acceleration Z(g)'].str.replace(',', '.').astype(float)
#_# ANGELS
df['Angle X(°)'] = df['Angle X(°)'].str.replace(',', '.').astype(float)
df['Angle Y(°)'] = df['Angle Y(°)'].str.replace(',', '.').astype(float)
df['Angle Z(°)'] = df['Angle Z(°)'].str.replace(',', '.').astype(float)
# ANGULAR VELOSITY
df['Angular velocity X(°/s)'] = df['Angular velocity X(°/s)'].str.replace(',', '.').astype(float)
df['Angular velocity Y(°/s)'] = df['Angular velocity Y(°/s)'].str.replace(',', '.').astype(float)
df['Angular velocity Z(°/s)'] = df['Angular velocity Z(°/s)'].str.replace(',', '.').astype(float)

# Alle numerischen Spalten mit Kommas in Punkten umwandeln
# df = df.replace({',': '.'}, regex=True).astype(float)

# print(df)

            Time Device name  Chip Time() Acceleration X(g) Acceleration Y(g)  \
0   09:51:45.800       COM12          NaN            -0,021             0,017   
1   09:51:45.900       COM12          NaN            -0,021             0,017   
2   09:51:46.000       COM12          NaN            -0,021             0,018   
3   09:51:46.099       COM12          NaN            -0,021             0,017   
4   09:51:46.199       COM12          NaN            -0,021             0,017   

  Acceleration Z(g) Angular velocity X(°/s) Angular velocity Y(°/s)  \
0             1,008                   0,000                   0,000   
1             1,008                   0,000                   0,000   
2             1,008                   0,000                   0,000   
3             1,008                   0,000                   0,000   
4             1,009                   0,000                   0,000   

  Angular velocity Z(°/s) Angle X(°)  ... Angle of course(°) GPS height(m)  \
0       

### eigentliche Zeit-Differenz-Berechnung

In [56]:
import numpy as np
from datetime import datetime

time = df['Time'].values  # Zeitspalte falls vorhanden

# Dein NumPy-Array mit Zeitstempeln als Strings
# time = np.array([' 18:06:05.104', ' 18:06:05.247', ' 18:06:05.304', ' 18:06:05.403'])

# Entferne führende und nachfolgende Leerzeichen aus der 'Time'-Spalte
df['Time'] = df['Time'].str.strip()
 
# Konvertiere die 'Time'-Spalte in das Datetime-Format
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S.%f')

# Zeitdifferenzen berechnen und in einer neuen Spalte speichern
df['Delta_t'] = df['Time'].diff().dt.total_seconds()
 
# print(df['Delta_t'])

Berechnung der Rotationsmatrix:

In [3]:
import numpy as np

def rotation_matrix(roll, pitch, yaw):
    roll = np.deg2rad(roll)
    pitch = np.deg2rad(pitch)
    yaw = np.deg2rad(yaw)

    R_x = np.array([[1, 0, 0],
                    [0, np.cos(roll), -np.sin(roll)],
                    [0, np.sin(roll), np.cos(roll)]])

    R_y = np.array([[np.cos(pitch), 0, np.sin(pitch)],
                    [0, 1, 0],
                    [-np.sin(pitch), 0, np.cos(pitch)]])

    R_z = np.array([[np.cos(yaw), -np.sin(yaw), 0],
                    [np.sin(yaw), np.cos(yaw), 0],
                    [0, 0, 1]])

    R = R_z @ R_y @ R_x
    return R

Transformiere die Beschleunigungsdaten:

Berechne die globale Beschleunigung, indem du die lokale Beschleunigung mit der Rotationsmatrix multiplizierst:

In [48]:
def transform_acceleration(acc_local, roll, pitch, yaw):
    R = rotation_matrix(roll, pitch, yaw)
    acc_global = R @ acc_local
    return acc_global

Integration zur Positionsbestimmung
Berechne die Geschwindigkeit und Position: Nach der Transformation der Beschleunigung in das globale Koordinatensystem, kannst du die Geschwindigkeit und die Position durch Integration berechnen. Dazu benötigst du die Zeitdifferenz (Delta_t), um die Beschleunigung in Geschwindigkeit und dann die Geschwindigkeit in Position zu integrieren.

Berechnung der Geschwindigkeit und Position:

In [49]:
def integrate(accelerations, delta_t):
    velocity = np.cumsum(accelerations * delta_t)
    position = np.cumsum(velocity * delta_t)
    return velocity, position


Berücksichtigung der Erdbeschleunigung

In [50]:
def adjust_for_gravity(acc_global, gravity=9.81):
    acc_global[2] -= gravity  # Annahme: Z-Achse ist vertikal
    return acc_global


### Lese die Daten aus dem DataFrame und wende die Funktionen an

In [ ]:
import numpy as np
# Initialisiere Listen für Geschwindigkeit und Position
# velocity_x, velocity_y, velocity_z = [], [], []
# position_x, position_y, position_z = [], [], []
velocity_x, velocity_y, velocity_z = np.zeros(len(df)), np.zeros(len(df)), np.zeros(len(df))
position_x, position_y, position_z = np.zeros(len(df)), np.zeros(len(df)), np.zeros(len(df))



# Iteriere über die Zeilen des DataFrames
for index, row in df.iterrows():
    delta_t = row['Delta_t']
    acc_x = row['Acceleration X(g)']
    acc_y = row['Acceleration Y(g)']
    acc_z = row['Acceleration Z(g)']
    angle_x = row['Angle X(°)']
    angle_y = row['Angle Y(°)']
    angle_z = row['Angle Z(°)']

    # Lokale Beschleunigung in ein numpy Array
    acc_local = np.array([acc_x, acc_y, acc_z])  

    # Transformiere die lokale Beschleunigung in die globale Koordinate
    acc_global = transform_acceleration(acc_local, angle_x, angle_y, angle_z)

    # Berücksichtige die Erdbeschleunigung
    acc_global = adjust_for_gravity(acc_global)

    # Berechne Geschwindigkeit und Position
    velocity, position = integrate(acc_global, delta_t)

    # Füge die berechneten Werte zu den Listen hinzu
    velocity_x.append(velocity[0])
    velocity_y.append(velocity[1])
    velocity_z.append(velocity[2])
    position_x.append(position[0])
    position_y.append(position[1])
    position_z.append(position[2])    


    # Füge die berechneten Werte dem DataFrame hinzu
    # df['Velocity X(m/s)'] = velocity_x
    # df['Velocity Y(m/s)'] = velocity_y
    # df['Velocity Z(m/s)'] = velocity_z
    # df['Position X(m)'] = position_x
    # df['Position Y(m)'] = position_y
    # df['Position Z(m)'] = position_z

    # Speichere die Ergebnisse in eine neue CSV-Datei
    # df.to_csv('sensor_data_with_position.csv', index=False)

      

In [57]:
import numpy as np
import pandas as pd

def rotation_matrix(roll, pitch, yaw):
    roll = np.deg2rad(roll)
    pitch = np.deg2rad(pitch)
    yaw = np.deg2rad(yaw)

    R_x = np.array([[1, 0, 0],
                    [0, np.cos(roll), -np.sin(roll)],
                    [0, np.sin(roll), np.cos(roll)]])

    R_y = np.array([[np.cos(pitch), 0, np.sin(pitch)],
                    [0, 1, 0],
                    [-np.sin(pitch), 0, np.cos(pitch)]])

    R_z = np.array([[np.cos(yaw), -np.sin(yaw), 0],
                    [np.sin(yaw), np.cos(yaw), 0],
                    [0, 0, 1]])

    R = R_z @ R_y @ R_x
    return R

def transform_acceleration(acc_local, roll, pitch, yaw):
    R = rotation_matrix(roll, pitch, yaw)
    acc_global = R @ acc_local
    return acc_global

def adjust_for_gravity(acc_global, gravity=9.81):
    acc_global[2] -= gravity  # Annahme: Z-Achse ist vertikal
    return acc_global

def integrate(accelerations, delta_t):
    velocity = np.cumsum(accelerations * delta_t)
    position = np.cumsum(velocity * delta_t)
    return velocity, position

# Beispiel DataFrame laden
# data = pd.read_csv('sensor_data.csv')  # Ersetze 'sensor_data.csv' durch deinen Dateipfad

# Initialisiere Arrays für Geschwindigkeit und Position
num_rows = len(df)
velocity_x = np.zeros(num_rows)
velocity_y = np.zeros(num_rows)
velocity_z = np.zeros(num_rows)
position_x = np.zeros(num_rows)
position_y = np.zeros(num_rows)
position_z = np.zeros(num_rows)

# Iteriere über die Zeilen des DataFrames
for index, row in df.iterrows():
    delta_t = row['Delta_t']
    acc_x = row['Acceleration X(g)']
    acc_y = row['Acceleration Y(g)']
    acc_z = row['Acceleration Z(g)']
    angle_x = row['Angle X(°)']
    angle_y = row['Angle Y(°)']
    angle_z = row['Angle Z(°)']

    # Lokale Beschleunigung in ein numpy Array
    acc_local = np.array([acc_x, acc_y, acc_z])

    # Transformiere die lokale Beschleunigung in die globale Koordinate
    acc_global = transform_acceleration(acc_local, angle_x, angle_y, angle_z)
    
    # Berücksichtige die Erdbeschleunigung
    acc_global = adjust_for_gravity(acc_global)

    # Berechne Geschwindigkeit und Position (außer für den ersten Schritt)
    if index == 0:
        velocity = np.zeros(3)
        position = np.zeros(3)
    else:
        velocity, position = integrate(acc_global, delta_t)

    # Speichere die berechneten Werte in den Arrays
    velocity_x[index] = velocity[0]
    velocity_y[index] = velocity[1]
    velocity_z[index] = velocity[2]
    position_x[index] = position[0]
    position_y[index] = position[1]
    position_z[index] = position[2]

# Füge die berechneten Werte dem DataFrame hinzu
df['Velocity X(m/s)'] = velocity_x
df['Velocity Y(m/s)'] = velocity_y
df['Velocity Z(m/s)'] = velocity_z
df['Position X(m)'] = position_x
df['Position Y(m)'] = position_y
df['Position Z(m)'] = position_z

# Speichere die Ergebnisse in eine neue CSV-Datei
df.to_csv('sensor_data_with_position2.csv', index=False, decimal=',')



Neuer Ansatz

In [53]:
import numpy as np
import pandas as pd

def rotation_matrix(roll, pitch, yaw):
    roll = np.deg2rad(roll)
    pitch = np.deg2rad(pitch)
    yaw = np.deg2rad(yaw)

    R_x = np.array([[1, 0, 0],
                    [0, np.cos(roll), -np.sin(roll)],
                    [0, np.sin(roll), np.cos(roll)]])

    R_y = np.array([[np.cos(pitch), 0, np.sin(pitch)],
                    [0, 1, 0],
                    [-np.sin(pitch), 0, np.cos(pitch)]])

    R_z = np.array([[np.cos(yaw), -np.sin(yaw), 0],
                    [np.sin(yaw), np.cos(yaw), 0],
                    [0, 0, 1]])

    R = R_z @ R_y @ R_x
    return R

def transform_acceleration(acc_local, roll, pitch, yaw):
    R = rotation_matrix(roll, pitch, yaw)
    acc_global = R @ acc_local
    return acc_global

def adjust_for_gravity(acc_global, gravity=9.81):
    acc_global[2] -= gravity  # Annahme: Z-Achse ist vertikal
    return acc_global

def integrate(accelerations, delta_t):
    velocity = np.zeros_like(accelerations)
    position = np.zeros_like(accelerations)
    for i in range(1, len(accelerations)):
        velocity[i] = velocity[i-1] + accelerations[i] * delta_t[i]
        position[i] = position[i-1] + velocity[i] * delta_t[i]
    return velocity, position

# Beispiel DataFrame laden
# data = pd.read_csv('sensor_data.csv')  # Ersetze 'sensor_data.csv' durch deinen Dateipfad
df = pd.read_csv('sensor_data.csv', decimal=',')  # Beispielhafte CSV-Lesemethode

# Offsets bestimmen (Beispielwerte, bitte anpassen)
offset_x = 0.0  # Offset für Beschleunigung X (in g)
offset_y = 0.0  # Offset für Beschleunigung Y (in g)
offset_z = 0.0  # Offset für Beschleunigung Z (in g)
offset_roll = 0.0  # Offset für Rollwinkel (in °)
offset_pitch = 0.0  # Offset für Pitchwinkel (in °)
offset_yaw = 0.0  # Offset für Yawwinkel (in °)

# Initialisiere Arrays für Geschwindigkeit und Position
num_rows = len(df)
velocity_x = np.zeros(num_rows)
velocity_y = np.zeros(num_rows)
velocity_z = np.zeros(num_rows)
position_x = np.zeros(num_rows)
position_y = np.zeros(num_rows)
position_z = np.zeros(num_rows)

# Listen für Beschleunigung und Delta_t
accel_x = np.zeros(num_rows)
accel_y = np.zeros(num_rows)
accel_z = np.zeros(num_rows)
delta_t = np.zeros(num_rows)

# Iteriere über die Zeilen des DataFrames
for index, row in df.iterrows():
    delta_t[index] = row['Delta_t']
    accel_x[index] = row['Acceleration X(g)'] - offset_x
    accel_y[index] = row['Acceleration Y(g)'] - offset_y
    accel_z[index] = row['Acceleration Z(g)'] - offset_z
    angle_x = row['Angle X(°)'] - offset_roll
    angle_y = row['Angle Y(°)'] - offset_pitch
    angle_z = row['Angle Z(°)'] - offset_yaw

    # Lokale Beschleunigung in ein numpy Array
    acc_local = np.array([accel_x[index], accel_y[index], accel_z[index]])

    # Transformiere die lokale Beschleunigung in die globale Koordinate
    acc_global = transform_acceleration(acc_local, angle_x, angle_y, angle_z)
    
    # Berücksichtige die Erdbeschleunigung
    acc_global = adjust_for_gravity(acc_global)

    # Speichere die berechneten Werte in den Arrays
    accel_x[index], accel_y[index], accel_z[index] = acc_global

# Berechne Geschwindigkeit und Position
velocity, position = integrate(np.column_stack([accel_x, accel_y, accel_z]), delta_t)

# Füge die berechneten Werte dem DataFrame hinzu
df['Velocity X(m/s)'] = velocity[:, 0]
df['Velocity Y(m/s)'] = velocity[:, 1]
df['Velocity Z(m/s)'] = velocity[:, 2]
df['Position X(m)'] = position[:, 0]
df['Position Y(m)'] = position[:, 1]
df['Position Z(m)'] = position[:, 2]

# Speichere die Ergebnisse in eine neue CSV-Datei
df.to_csv('sensor_data_with_position2.csv', index=False, decimal=',')


Darstellung mit Plot

In [58]:
import plotly.graph_objects as go
import pandas as pd

# Beispiel DataFrame laden (ersetze df durch deinen DataFrame)
data = df

# Beschneide die Daten ab Index 5
data_subset = data.iloc[5:]

# Erstelle eine Plotly-Figur für 3D
fig = go.Figure()

# Füge die 3D-Linie hinzu
fig.add_trace(go.Scatter3d(
    x=data_subset['Position X(m)'],
    y=data_subset['Position Y(m)'],
    z=data_subset['Position Z(m)'],
    mode='lines',
    line=dict(width=2, color='blue'),
    marker=dict(size=5, color=data_subset.index, colorscale='Viridis', showscale=True)
))

# Finde die Grenzen für die x-, y- und z-Achsen
x_range = [data_subset['Position X(m)'].min(), data_subset['Position X(m)'].max()]
y_range = [data_subset['Position Y(m)'].min(), data_subset['Position Y(m)'].max()]
z_range = [data_subset['Position Z(m)'].min(), data_subset['Position Z(m)'].max()]

# Berechne die gleiche Skalierung für alle Achsen
max_range = max(
    x_range[1] - x_range[0], 
    y_range[1] - y_range[0],
    z_range[1] - z_range[0]
)
x_center = (x_range[1] + x_range[0]) / 2
y_center = (y_range[1] + y_range[0]) / 2
z_center = (z_range[1] + z_range[0]) / 2

# Setze die gleichen Grenzen für die x-, y- und z-Achsen
fig.update_layout(
    title='3D-Positionsdaten des Sensors als Linie ab Index 5',
    scene=dict(
        xaxis_title='Position X (m)',
        yaxis_title='Position Y (m)',
        zaxis_title='Position Z (m)',
        xaxis=dict(range=[x_center - max_range/2, x_center + max_range/2]),
        yaxis=dict(range=[y_center - max_range/2, y_center + max_range/2]),
        zaxis=dict(range=[z_center - max_range/2, z_center + max_range/2])
    ),
    template='plotly_dark'  # Optional: Setzt ein dunkles Layout
)

# Zeige die Figur an
fig.show()
# Speichere die Figur als HTML-Datei
fig.write_html('3d_plot.html')





In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Beispiel DataFrame laden
data = df

# Beschneide die Daten ab Index 5
data_subset = data.iloc[5:]

# Erstelle eine 3D-Figur
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plotte die Daten ab Index 5
ax.plot(data_subset['Position X(m)'], data_subset['Position Y(m)'], data_subset['Position Z(m)'], marker='o', linestyle='-', color='b')

# Füge Titel und Achsenbeschriftungen hinzu
ax.set_title('3D-Positionsdaten des Sensors ab Index 5')
ax.set_xlabel('Position X (m)')
ax.set_ylabel('Position Y (m)')
ax.set_zlabel('Position Z (m)')

# Zeige das Diagramm an
plt.show()

